In [5]:
%matplotlib inline 

from fnmatch import fnmatch
import urllib, urllib2, cookielib
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#import re
import requests
from pattern import web
#from matplotlib import rcParams


##INPUTS##
leagueid=732290
year=2016
weeks=6
returnmax=True #True outputs results in a csv file of teams, their weekly scores, and weekly maximum scores(replacing anyone in their lineup with a higher score on the bench). 0 just outputs a csv file of weekly scores for each team
rostersize=9
benchsize=6
##########


positions=('QB','RB','RB','WR','WR','TE','FLEX','D','K','Total')
#gets the list of teams in an array
def team_list():
    url="http://games.espn.com/ffl/leagueoffice?leagueId="+str(leagueid)+"&seasonId="+str(year)
    xml=requests.get(url).text
    if str(web.Element(xml).by_tag('head')[0]).find('Log In - ')>0:
        print "***league isn't public, can't access data***"
        return
    dom = web.Element(xml).by_tag('li')
    lis=[]
    
    i=0
    while len(str(dom[i+66]))>80:
        temp=dom[i+66]
        temp = str(temp)
        lis.append(temp[130:len(temp)-46])
        i=i+1
    return lis

#gets the teams wins
def get_wins(t):
    url="http://games.espn.com/ffl/clubhouse?leagueId="+str(leagueid)+"&teamId="+str(t)+"&seasonId="+str(year)
    xml=requests.get(url).text
    rec=xml[xml.find('Record'):xml.find('Record')+20]
    rec=rec[17:rec.find('-')]
    return rec

In [6]:
numteams=len(team_list())
#Returns an array of points scored by team t in week w. m is whether or not you want the weekly maximum points returned. The retunred pointed are in the order of ('QB','RB','RB','WR','WR','TE','FLEX','D','K','Total')
def get_points(t,w,m):
    if m:
        return get_maxpoints(t,w)
    else:
        return get_normalpoints(t,w)

def get_normalpoints(t,w):
    points=[0 for z in range(rostersize+1)] #the last one is their wins
    url="http://games.espn.com/ffl/boxscorequick?leagueId="+str(leagueid)+"&teamId="+str(t)+"&scoringPeriodId="+str(w)+"&seasonId="+str(year)+"&view=scoringperiod&version=quick"
    xml=requests.get(url).text
    dom = web.Element(xml).by_tag('tr')
    
    if len(dom)<20:
        return "noteam"
    
    for i in range(rostersize):
        temp=dom[4+i] #4 though 12 is QB to K, 15 to 20 is Bench
        if str(dom[4]).find('STARTERS')>0:
            temp=dom[6+i]
            
        if len(temp.by_tag('td'))<5:
            temp2=str(temp.by_tag('td')[len(temp.by_tag('td'))-1])
        else:
            temp2=str(temp.by_tag('td')[4]) #gets to the points section
        if temp2.find('--')>0:
            temp2=0
        else:
            scorestart=temp2.find('</td>')-14+temp2[(temp2.find('</td>')-15):temp2.find('</td>')].find('>')
            scoreend=temp2.find('</td>')-15+temp2[(temp2.find('</td>')-15):temp2.find('</td>')+1].find('<')
            temp2=temp2[scorestart:scoreend] #extracts the points
        points[i]=float(temp2)
        points[rostersize]=float(temp2)+points[rostersize]
    return points

def get_maxpoints(t,w):
    maxpoints=get_normalpoints(t,w)
    if maxpoints=="noteam":
        return "noteam"
    for i in range(6):
        tempscores=[0,0,0]
        benchscore=0
        url="http://games.espn.com/ffl/boxscorequick?leagueId="+str(leagueid)+"&teamId="+str(t)+"&scoringPeriodId="+str(w)+"&seasonId="+str(year)+"&view=scoringperiod&version=quick"
        xml=requests.get(url).text
        dom = web.Element(xml).by_tag('tr')
        temp=dom[15+i] #4 though 12 is QB to K, 15 to 20 is Bench
        if str(dom[15]).find('playerid')<0:
            temp=dom[17+i]

        if len(temp.by_tag('td'))<5:
            temp2=str(temp.by_tag('td')[len(temp.by_tag('td'))-1])
        else:
            temp2=str(temp.by_tag('td')[4]) #gets to the points section
        if temp2.find('--')>0:
            temp2=0
        else:
            scorestart=temp2.find('</td>')-14+temp2[(temp2.find('</td>')-15):temp2.find('</td>')].find('>')
            scoreend=temp2.find('</td>')-15+temp2[(temp2.find('</td>')-15):temp2.find('</td>')+1].find('<')
            temp2=temp2[scorestart:scoreend] #takes out the points
            benchscore=float(temp2)

        temp=str(temp)
        if temp.find('QB')>0:
            if maxpoints[0]<benchscore:
                maxpoints[0]=benchscore
        if temp.find('RB')>0:
            tempscores=[maxpoints[1],maxpoints[2],benchscore]
            tempscores=sorted(tempscores,reverse=True)
            maxpoints[1]=tempscores[0]
            maxpoints[2]=tempscores[1]
            if maxpoints[6]<tempscores[2]:
                maxpoints[6]=tempscores[2]
        if temp.find('WR')>0:
            tempscores=[maxpoints[3],maxpoints[4],benchscore]
            tempscores=sorted(tempscores,reverse=True)
            maxpoints[3]=tempscores[0]
            maxpoints[4]=tempscores[1]
            if maxpoints[6]<tempscores[2]:
                maxpoints[6]=tempscores[2]
        if temp.find('TE')>0:
            tempscores=[maxpoints[5],benchscore,0]
            tempscores=sorted(tempscores,reverse=True)
            maxpoints[5]=tempscores[0]
            if maxpoints[6]<tempscores[1]:
                maxpoints[6]=tempscores[1]
        if temp.find('D/ST')>0:
            if maxpoints[7]<benchscore:
                maxpoints[7]=benchscore
        if temp.find('nbsp;K')>0:
            if maxpoints[7]<benchscore:
                maxpoints[7]=benchscore
    maxpoints[rostersize]=0
    for i in range(9):
        maxpoints[rostersize]=maxpoints[rostersize]+maxpoints[i]                  
    return maxpoints

In [7]:
#MAKES EXCELL FILE OF JUST TOTAL POINTS called HopkinsFF_pointsTotal with the year and weeks and whether it is weekly points or weekly maximum points at the end in .csv format
teams=team_list()
with open('Outputs\Points'+str(year)+"weeks"+str(weeks)+"maxpoints"+str(returnmax)+'.csv','w') as out_file:
    Header='Team'
    for week in range(weeks):
        Header+=',Week'+str(week+1)
    out_file.write(Header+',Wins\n')
    
    t=0
    i=0
    while (i <(len(teams)) and t<100):
        if get_points(t,1,returnmax)=="noteam":
            t=t+1
        else:
            out_string=str(teams[i])
            for week in range(weeks):
                temp=get_points(t,week+1,returnmax)
                out_string+=","+str(temp[rostersize])
            out_string+=","+str(get_wins(t))
            out_string+="\n"
            t=t+1
            i=i+1
            print "Completed Team: " +str(i)
            out_file.write(out_string)
print "DONE"

Completed Team: 1
Completed Team: 2
Completed Team: 3
Completed Team: 4
Completed Team: 5
Completed Team: 6
Completed Team: 7
Completed Team: 8
Completed Team: 9
Completed Team: 10
Completed Team: 11
Completed Team: 12
DONE
